# RAG

- "RAG"는 "Retrieval-Augmented Generation"의 약자로, "검색-증강 생성"이라는 의미를 가지고 있습니다. 이는 자연어 처리(NLP) 및 기계 학습 분야, 특히 챗봇이나 질문-응답 시스템과 같은 고급 언어 모델을 구축하는 데 사용되는 기술입니다.

- RAG에 대한 간략한 개요는 다음과 같습니다:

검색과 생성의 결합: RAG는 NLP의 두 가지 주요 구성 요소인 정보 검색과 응답 생성을 결합합니다. 검색 부분은 관련 정보를 찾기 위해 대규모 데이터베이스나 문서 컬렉션을 검색하는 과정을 포함합니다. 생성 부분은 검색된 정보를 바탕으로 일관되고 맥락적으로 적절한 텍스트를 생성하는 과정입니다.


작동 방식: RAG 시스템에서 질문이나 프롬프트가 주어지면 모델은 먼저 질문에 대한 답변을 제공하는 데 유용한 정보를 포함할 수 있는 관련 문서나 텍스트를 검색합니다. 그런 다음 이 검색된 정보를 생성 모델에 공급하여 일관된 응답을 합성합니다.


- 장점: RAG의 주요 장점은 모델이 외부 지식을 활용할 수 있게 하여 보다 정확하고 상세하며 맥락적으로 관련된 답변을 제공할 수 있다는 것입니다. 이는 특정 지식이나 사실적 정보가 필요한 질문에 특히 유용합니다.


- 응용 분야: RAG는 챗봇, 질문-응답 시스템 및 정확하고 맥락적으로 관련된 정보를 제공하는 것이 중요한 다른 AI 도구와 같은 다양한 응용 분야에 사용됩니다. 특히 모델이 다양한 주제와 데이터를 기반으로 이해하고 응답을 생성해야 하는 상황에서 유용합니다.
개발 및 사용: AI 및 기계 학습 커뮤니티에서 RAG는 다양한 연구 논문과 구현이 개발되고 있으며 주요 초점 중 하나입니다. 이는 학습된 정보뿐만 아니라 외부 소스에서 새롭고 관련된 정보를 통합하여 응답의 질과 관련성을 향상시키는 더 정교한 AI 시스템으로 나아가는 단계를 나타냅니다.

In [16]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 53.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67

## Retrieval - RAG의 첫 번째 단계

### Data Loader 

- Loader는 소스에서 데이터를 추출하고 랭체인에 가져다 주는 코드 


### Splitters

- 문서에서 필요한 부분만 찾을 때 ! 

- 두 가지 방법이 있음. 


- from_tiktoken_encoder : 텍스트의 길이를 모델이 카운트 하는 방법 (모델이 자연어를 바라보는 방법. 토큰)


#### Embedding : 사람이 읽는 텍스트를 컴퓨터가 이해할 수 있는 숫자들로 변환하는 작업. 

- 단어를 표현하는 벡터로 만든다! 
- 단어를 차원들로 표현할 때 좋은 점은 다른 단어를 얻기 위해 가진 단어를 가지고 연산을 할 수 있음. 
- 단어를 연관시키고 싶다면 ? 
    - https://turbomaze.github.io/word2vecjson/ 이런 웹사이트에서 단어를 입력하면 연관된 단어를 찾거나 벡터 연산을 통해 비슷한 단어를 찾아준다. (참고)
    - https://www.youtube.com/watch?v=2eWuYf-aZE4 스포티파이에서 LLM을 어떻게 활용하나 ? 

- Embedding을 매번 코드를 실행할 때마다 하는 것은 좋은게 아니다. 한 번 document를 embed 하면 문서가 바뀔 때까지 embeddings도 바뀌지 않아야한다. 

- **Vector Store** : 일종의 데이터 벡스. 벡터 공간에서 검색을 할 수 있게 해준다. 벡터들을 만들고 나서 그것들을 캐싱해주고 vector stroe에 그 벡터들을 넣어주면, 우리가 검색을 할 수 있다. 
    - 다양한 vector store를 사용할 수 있다. (유료 무료 버전이 있다.)
    - Chroma 사용. 
        - Chroma에는 분할된(Splitted) 문서와 openAI embeddings model을 전달해야 한다. 
        - cache 폴더에 처음 embedding한 값들이 저장된다.


- langsmith : 시각화 툴. 베타 서비스 중이라서 가입후 웨이팅 해야함. (https://smith.langchain.com/o/3a7fde8e-2b57-5c01-aae7-c830f9fc466c/settings) 

- API key 생성 후 .env 파잃에 OPEN_API_KEY 등록.
- 데이터의 흐름을 파악하고 이해할 수 있다. 

In [1]:
from typing import Text
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)# 문장이나 문단 단위로 자를 수 있다 !

loader = UnstructuredFileLoader("./files/chapter1.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

/home/minyoungxi/.local/share/virtualenvs/LLM_Langchain_APP-7aD6jX0r/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/minyoungxi/.local/share/virtualenvs/LLM_Langchain_APP-7aD6jX0r/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [13]:
# Embedding 예시

from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

vector = embedder.embed_documents([
    "hi",
    "how",
    "are",
    "you longer sentences because"
])

print(len(vector), len(vector[0])) # 단어의 개수, 벡터의 차원


4 1536


## RetirevalQA

"RetrievalQA" 기능은 특히 질문에 대한 답변을 찾기 위해 외부의 문서나 데이터베이스에서 정보를 검색(retrieval)하는 과정을 자동화하는 데 중점을 두고 있습니다.

RetrievalQA은 개념이 모호해서 공식문서에 설명이 없다

- 공식문서에는 LangChain Expression Language(LCEL)을 사용할 것을 권장합니다. 
- LCEL은 어렵다 ! 

- Stuff : 우리가 찾은 document들로 prompt를 stuff(채우기) 하는데 사용한다는 뜻.

- retriver는 class의 interface 입니다. document를 많은 장소로부터 Retrieve(선별하여 가져오기)할 수 있습니다. vector store말고 다른 곳에서도. (database, cloud도 가능)

In [3]:
from typing import Text
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)# 문장이나 문단 단위로 자를 수 있다 !

loader = UnstructuredFileLoader("./files/chapter1.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever = vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions")

/home/minyoungxi/.local/share/virtualenvs/LLM_Langchain_APP-7aD6jX0r/lib/python3.10/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


"Victory Mansions is a building with glass doors and a hallway that smells of boiled cabbage and old rag mats. It has a large colored poster of a man's face with a black mustache on one of the walls. The flat that Winston Smith lives in is located on the seventh floor and the building has a non-functioning lift. The walls of the flat have a telescreen, which is an oblong metal plaque that cannot be completely shut off and constantly broadcasts information. The building is located in a cold and colorless world, with little eddies of wind whirling dust and torn paper in the streets. There are posters of the black-mustachioed face plastered everywhere, including on the house-front opposite Victory Mansions."